In [1]:
import numpy as np
import pandas as pd

In [2]:
account_static = pd.read_csv('./data/账户静态信息.csv')
account_trade = pd.read_csv('./data/账户交易信息.csv')
y_train = pd.read_csv('./data/训练集标签.csv')
y_test = pd.read_csv('./data/test_dataset.csv')

In [5]:
account_trade.columns

Index(['jylsxh', 'zhdh', 'dfzh', 'jdbj', 'jyje', 'zhye', 'dfhh', 'jyrq',
       'jysj', 'jyqd', 'zydh', 'dfmccd'],
      dtype='object')

In [18]:
account_trade['jyje'].astype(str).str.split('.').str[1].unique()

array(['0', '99', '98', '96', '94', '95', '76', '91', '63', '9', '88',
       '42', '77', '11', '21', '81', '46', '51', '5', '01', '62', '7',
       '3', '86', '35', '89', '06', '1', '44', '43', '36', '55', '97',
       '48', '6', '69', '37', '25', '82', '22', '24', '52', '58', '33',
       '45', '49', '56', '65', '41', '75', '02', '61', '47', '87', '4',
       '73', '08', '84', '13', '59', '57', '83', '93', '26', '05', '72',
       '85', '17', '78', '67', '66', '38', '39', '8', '64', '53', '12',
       '92', '15', '04', '03', '2', '74', '23', '14', '19', '07', '16',
       '79', '68', '09', '18', '32', '31', '34', '71', '28', '29', '27',
       '54'], dtype=object)

In [23]:
account_trade.columns

Index(['jylsxh', 'zhdh', 'dfzh', 'jdbj', 'jyje', 'zhye', 'dfhh', 'jyrq',
       'jysj', 'jyqd', 'zydh', 'dfmccd'],
      dtype='object')

In [31]:
account_trade[account_trade['jdbj'] == 0].groupby(['zhdh', 'jyqd']).size().reset_index().groupby('zhdh')\
        .apply(lambda x: x.sort_values(by=0).iloc[-1]['jyqd'])\
        .reset_index().rename(columns={0: f'most_jyqd'})

,zhdh,most_jyqd
0,00037295453A928A,E96ED478
1,0004CC075464D54B,E96ED478
2,000AA77144DC1BCC,E96ED478
3,001ABBF3373AFC5B,757B505C
4,00310769938BC172,091D584F
...,...,...
5992,FFC065BECA5E133C,091D584F
5993,FFD2E9065F19B38D,E96ED478
5994,FFD48BA98FA2D299,757B505C
5995,FFD8F2A4DB42AC6D,091D584F


In [32]:
account_trade[account_trade['jdbj'] == 1].groupby(['zhdh', 'jyqd']).size().reset_index().groupby('zhdh')\
        .apply(lambda x: x.sort_values(by=0).iloc[-1]['jyqd'])\
        .reset_index().rename(columns={0: f'most_jyqd'})

,zhdh,most_jyqd
0,00037295453A928A,757B505C
1,0004CC075464D54B,621461AF
2,000AA77144DC1BCC,E96ED478
3,001ABBF3373AFC5B,621461AF
4,00310769938BC172,621461AF
...,...,...
5993,FFC065BECA5E133C,274AD478
5994,FFD2E9065F19B38D,E96ED478
5995,FFD48BA98FA2D299,621461AF
5996,FFD8F2A4DB42AC6D,621461AF
